# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

import sys
sys.path.append('../')

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

from IPython.display import Markdown

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 64 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) is None:
            doc.abstract = paper['abstract']
            
        doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
        doc.highlight_authors_in_list(hl_list)

        full_md = html + doc.generate_markdown_text()
        # Make CSS paper specific
        ## Set CSS and figure div names accordingly
        divref = paper_id.replace('.', '_')
        full_md = full_md.replace('fig1', f'div_{divref}_fig1')\
                         .replace('fig2', f'div_{divref}_fig2')\
                         .replace('fig3', f'div_{divref}_fig3')
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.00635/dr2_data.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:282: LatexWarning: Latex injecting: 'flux' from 'tmp_2204.00635/flux.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:282: LatexWarning: Latex injecting: 'spix' from 'tmp_2204.00635/spix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:282: LatexWarning: Latex injecting: 'sample1' from 'tmp_2204.00635/sample1.tex'
  warnings.warn(LatexWar

Retrieving document from  https://arxiv.org/e-print/2204.01256


/var/folders/jb/twt_8kfx4mj86m9v8jgccs_r5gbl99/T/ipykernel_43944/3844102717.py:37: LatexWarning: 2204.01256 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.01330/HELIOS-R2_brown_dwarfs_1.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


Retrieving document from  https://arxiv.org/e-print/2204.01377


/var/folders/jb/twt_8kfx4mj86m9v8jgccs_r5gbl99/T/ipykernel_43944/3844102717.py:37: LatexWarning: 2204.01377 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.01421/MN-21-4463-MJ.R2.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.01594/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/i

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        if color in ('red',):
            display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.00793-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.00793) | **Spatially resolved molecular interstellar medium in a $z=6.6$ quasar  host galaxy**  |
|| Jianan Li, et al. -- incl., <mark>Fabian Walter</mark> |
|*Comments*| *12 pages, 6 figures, accepted for publication in ApJ*|
|**Abstract**| We present high spatial resolution (0.4", 2.2kpc) observations of the CO(6-5), CO(7-6) and [CI] lines and dust continuum emission from the interstellar medium in the host galaxy of the quasar J0305$-$3150 at $z=6.6$. These, together with archival [CII] data at comparable spatial resolution, enable studies of the spatial distribution and kinematics between the ISM in different phases. When comparing the radial profiles of CO, [CII] and the dust continuum, we find that the CO and dust continuum exhibit similar spatial distributions, both of which are less extended than the [CII], indicating that the CO and dust continuum are tracing the same gas component, while the [CII] is tracing a more extended one. In addition, we derive the radial profiles of the [CII]/CO, [CII]/far-infrared (FIR), CO/FIR, and dust continuum $S_{98.7 \rm GHz}/S_{258.1 \rm GHz}$ ratios. We find a decreasing $S_{98.7 \rm GHz}/S_{258.1 \rm GHz}$ ratio with radius, possibly indicating a decrease of dust optical depth with increasing radius. We also detect some of the ISM lines and continuum emission in the companion galaxies previously discovered in the field around J0305$-$3150. Through comparing the line-to-line and line-to-FIR ratios, we find no significant differences between the quasar and its companion galaxies. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.01245-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.01245) | **Staring at the Shadows of Archaic Galaxies: Damped Ly$α$ and Metal  Absorbers toward a Young $z \sim 6$ Weak-line Quasar**  |
|| <mark>Irham Taufik Andika</mark>, et al. -- incl., <mark>Knud Jahnke</mark>, <mark>Eduardo Bañados</mark>, <mark>Sarah E. I. Bosman</mark>, <mark>Frederick B. Davies</mark> |
|*Comments*| *22 pages, 12 figures, and 1 table. Accepted for publication in The Astronomical Journal. We welcome the comments from the reader. Related paper: arXiv:2009.07784*|
|**Abstract**| We characterize the Ly$\alpha$ halo and absorption systems toward PSO J083+11, a unique $z=6.3401$ weak-line quasar, using Gemini/GNIRS, Magellan/FIRE, and VLT/MUSE data. Strong absorptions by hydrogen and several metal lines (e.g., CII, MgII, and OI) are discovered in the spectrum, which indicates the presence of: (i) a proximate sub-damped Ly$\alpha$ (sub-DLA) system at $z=6.314$ and (ii) a MgII absorber at $z=2.2305$. To describe the observed damping wing signal, we model the Ly$\alpha$ absorption with a combination of a sub-DLA with the neutral hydrogen column density of $\log N_\mathrm{HI} = 20.03 \pm 0.30$ cm$^{-2}$ and absorption from the intergalactic medium with a neutral fraction of around 10 percent. The sub-DLA toward PSO J083+11 has an abundance ratio of [C/O] $=-0.04 \pm 0.33$ and metallicity of [O/H] $=-2.19 \pm 0.44$, similar to those of low-redshift metal-poor DLAs. These measurements suggest that the sub-DLA might truncate PSO J083+11's proximity zone size and complicate the quasar lifetime measurement. However, this quasar shows no sign of a Ly$\alpha$ halo in the MUSE datacube, where the estimated $1\sigma$ limit of surface brightness is $2.76 \times 10^{-18}$ erg s$^{-1}$ cm$^{-2}$ arcsec$^{-2}$ at aperture size of 1 arcsecond, or equivalent to a Ly$\alpha$ luminosity of $\leq 43.46$ erg s$^{-1}$. This non-detection, while being only weak independent evidence on its own, is at least consistent with a young quasar scenario, as expected for a quasar with a short accretion timescale. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.01256-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.01256) | **Monte Carlo Physarum Machine: Characteristics of Pattern Formation in  Continuous Stochastic Transport Networks**  |
|| Oskar Elek, <mark>Joseph N. Burchett</mark>, J. Xavier Prochaska, Angus G. Forbes |
|*Comments*| **|
|**Abstract**| We present Monte Carlo Physarum Machine: a computational model suitable for reconstructing continuous transport networks from sparse 2D and 3D data. MCPM is a probabilistic generalization of Jones's 2010 agent-based model for simulating the growth of Physarum polycephalum slime mold. We compare MCPM to Jones's work on theoretical grounds, and describe a task-specific variant designed for reconstructing the large-scale distribution of gas and dark matter in the Universe known as the Cosmic web. To analyze the new model, we first explore MCPM's self-patterning behavior, showing a wide range of continuous network-like morphologies -- called "polyphorms" -- that the model produces from geometrically intuitive parameters. Applying MCPM to both simulated and observational cosmological datasets, we then evaluate its ability to produce consistent 3D density maps of the Cosmic web. Finally, we examine other possible tasks where MCPM could be useful, along with several examples of fitting to domain-specific data as proofs of concept. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.01377-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.01377) | **Galactic Cosmic Rays at Mars and Venus: Temporal Variations from Hours  to Decades Measured as the Background Signal of Onboard Micro-Channel Plates**  |
|| Yoshifumi Futaana, et al. -- incl., <mark>Herman Andersson</mark> |
|*Comments*| *Submitted to ApJ (2022-03-19)*|
|**Abstract**| A Micro-Channel Plate (MCP) is a widely used component for counting particles in space. Using the background counts of MCPs on Mars Express and Venus Express orbiters operated over 17 years and 8 years, respectively, we investigate the galactic cosmic ray (GCR) characteristics in the inner solar system. The MCP background counts at Mars and Venus on a solar cycle time scale exhibit clear anti-correlation to the sunspot number. We conclude that the measured MCP background contain the GCR information. The GCR characteristics measured using the MCP background at Mars show features that are consistent with the ground-based measurement in solar cycle 24. The time lag between the sunspot number and the MCP background at Mars is found ~9 months. The shorter-term background data recorded along the orbits (with a time scale of several hours) also show evident depletion of the background counts due to the absorption of the GCR particles by the planets. Thanks to the visible planetary size change along an orbit, the GCR contribution to the MCP background can be separated from the internal contribution due to the \b{eta}-decay. Our statistical analysis of the GCR absorption signatures at Mars implies that the effective absorption size of Mars for the GCR particles have a >100 km larger radius than the solid Martian body. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", 'exports')

exported in  exports/2204.00793.md
    + exports/tmp_2204.00793/./cii_spectra.png
    + exports/tmp_2204.00793/./comp1.png
    + exports/tmp_2204.00793/./co65_spectra.png
    + exports/tmp_2204.00793/./comp2.png
    + exports/tmp_2204.00793/./co76_spectra.png
    + exports/tmp_2204.00793/./comp3.png
    + exports/tmp_2204.00793/./radial_profile.png
    + exports/tmp_2204.00793/./radial_ratio_co_cii.png
    + exports/tmp_2204.00793/./figure_sflaw.png
    + exports/tmp_2204.00793/./figure_deficit.png
exported in  exports/2204.01245.md
    + exports/tmp_2204.01245/figures/fig_spec_model.png
    + exports/tmp_2204.01245/figures/fig_velo_plot_first.png
    + exports/tmp_2204.01245/figures/fig_velo_plot_last.png
    + exports/tmp_2204.01245/figures/fig_halo_flux.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];


<style>
#wrap{ overflow:auto; }
#div_2204_00793_fig1{ width:100%; float:left; padding: 5px;  }
#div_2204_00793_fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#div_2204_00793_fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\hii}{\ion{H}{2}}$
$\newcommand{\oiii}{[\ion{O}{3}]_{\rm 88 \mu m}}$
$\newcommand{\oi}{[\ion{O}{1}]_{\rm 146 \mu m}}$
$\newcommand{\lsun}{L_{\sun}}$
$\newcommand{\nele}{n_{e}}$
$\newcommand{\pcubcm}{\rm cm^{-3}}$
$\newcommand{\cotwo}{CO(2-1)}$
$\newcommand{\coone}{CO(1-0)}$
$\newcommand{\ci}{[\ion{C}{1}]_{\rm 369 \mu m} }$
$\newcommand{\cii}{[\ion{C}{2}]_{\rm 158 \mu m} }$
$\newcommand{\cosix}{CO(6-5) }$
$\newcommand{\cosev}{CO(7-6) }$
$\newcommand{\qso}{J0305-3150}$
$\newcommand{\kmps}{\rm km s^{-1}}$
$\newcommand{\mjypb}{\rm mJy  beam^{-1}}$
$\newcommand{\ujypb}{\rm \mu Jy  beam^{-1}}$
$\newcommand{\jykmps}{\rm Jy km s^{-1} }$
$\newcommand{\msun}{\rm M_{\sun}}$</div>

# Spatially resolved molecular interstellar medium in a z=6.6 quasar host galaxy

 [![arXiv](https://img.shields.io/badge/arXiv-2204.00793-b31b1b.svg)](https://arxiv.org/abs/2204.00793) _12 pages, 6 figures, accepted for publication in ApJ_

Jianan Li, et al. -- incl., <mark>Fabian Walter</mark> 

 **Abstract:** We present high spatial resolution ($\sim$0$\farcs$4, 2.2kpc) observations of the$\cosix$,$\cosev$and$\ci$lines and dust continuum emission from the interstellar medium in the host galaxy of the quasar$\qso$at$z=6.6$.These, together with archival$\ci$idata at comparable spatial resolution, enable studies of the spatial distribution and kinematics between the ISM in different phases.When comparing the radial profiles of CO,$\ci$iand the dust continuum, we find that the CO and dust continuum exhibit similar spatial distributions, both of which are less extended than the$\ci$i, indicating that the CO and dust continuum are tracing the same gas component, while the[CII]158um is tracing a more extended one.In addition, we derive the radial profiles of the$\ci$i/CO,$\ci$i/far-infrared (FIR), CO/FIR, and dust continuum$S_{98.7 \rm GHz}/S_{258.1 \rm GHz}$ratios.We find a decreasing$S_{98.7 \rm GHz}/S_{258.1 \rm GHz}$ratio with radius, possibly indicating a decrease of dust optical depth with increasing radius.We also detect some of the ISM lines and continuum emission in the companion galaxies previously discovered in the field around$\qso$.  Through comparing the line-to-line and line-to-FIR ratios, we find no significant differences between the quasar and its companion galaxies.
<div id="div_2204_00793_fig1">

<img src="tmp_2204.00793/./cii_spectra.png" alt="Fig2.1" width="16%"/><img src="tmp_2204.00793/./comp1.png" alt="Fig2.2" width="16%"/><img src="tmp_2204.00793/./co65_spectra.png" alt="Fig2.3" width="16%"/><img src="tmp_2204.00793/./comp2.png" alt="Fig2.4" width="16%"/><img src="tmp_2204.00793/./co76_spectra.png" alt="Fig2.5" width="16%"/><img src="tmp_2204.00793/./comp3.png" alt="Fig2.6" width="16%"/>

**Figure 2. -** Left column: Spectra of $\ci$i,  $\cosix$, $\cosev$  for the quasar $\qso$. Data are shown in yellow histograms. The red solid lines are Gaussian profile fits to the spectra with the line centers fixed to the $\ci$i  redshift of $z=6.61391$\citep{venemans19}. The continuum has been subtracted for each of the spectra here displayed. Right column: $\ci$i  spectra for the companion galaxies C1, C2 and C3.  Data are shown in yellow histograms. The red solid lines are Gaussian profile fits to the spectra with the line centers fixed to the $\ci$i  redshift obtained from  $\ci$tet{venemans19}.\label{spectra3} The blue dashed lines indicate the noise at $\pm$ 1 $\sigma$ level.
The $\ci$i  spectrum of C3 is extracted from an aperture with a radius of 0$\farcs$6.
The spectral channel widths for the quasar and the companion galaxies are 35 $\kmps$.
For the source C3,  we present a stacked spectrum of the $\cosev$  and $\cosix$  lines extracted from the peak positions on the top left of the  $\ci$i  spectrum panel. (*spectra3*)

</div>
<div id="div_2204_00793_fig2">

<img src="tmp_2204.00793/./radial_profile.png" alt="Fig5.1" width="50%"/><img src="tmp_2204.00793/./radial_ratio_co_cii.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Left: Radial profiles of different gas and dust continuum tracers. The line fluxes are normalized to the peak pixel fluxes.
Right: Similar as the left panel, but showing the radio profiles of different line ratios.
\label{fig:radial} (*fig:radial*)

</div>
<div id="div_2204_00793_fig3">

<img src="tmp_2204.00793/./figure_sflaw.png" alt="Fig6.1" width="50%"/><img src="tmp_2204.00793/./figure_deficit.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Left:  SFR surface density $vs$ molecular gas surface density for $\qso$  and its companion galaxy C3 (the molecular gas surface densities of C1 and C2 are unable to determine).
Comparison samples include local spiral  ("local spirals") and starburst ("local starbursts") galaxies  from $\ci$tet{kennicutt19} and $\ci$tet{kennicutt21},  the z=4.69 quasar BRI 1202 ("BRI 1202 QSO") and its companion submillimeter galaxy ("BRI 1202  SMG") from $\ci$tet{wagg12} and $\ci$tet{jones16},  and the z=6.33 quasar J0100+2802("J0100") from $\ci$tet{wang19b}. The SFRs of $\qso$  and C3 are calculated using the same formula as that presented in  $\ci$tet{kennicutt21}.
Right: $\ci$i  deficit diagram for the quasar $\qso$  and its companion galaxy C2 and C3 (C1 is not included because of difficulties in constraining the source size).   Assuming the CO lines of C3 to be spatially unresolved, we use the beam size of $\cosix$  as an upper limit for the source size in the TIR surface density estimation of C3.
 "QSOs" and "companions" refer to z$\sim$6 quasar and companion galaxy samples collected from
 $\ci$tet{neelman19} and $\ci$tet{decarli17}.
The black solid line represents the $\ci$i  deficit relation observed in local LIRGs from
$\ci$tet{diaz17}. For the objects in this work and in the comparison samples, the total infrared (TIR) luminosities are estimated from the continuum flux density close to the $\ci$i   line frequency through assuming a modified black body model with temperature of 47K and emissivity of 1.6.
\label{fig:def} (*fig:def*)

</div>


<style>
#wrap{ overflow:auto; }
#div_2204_01245_fig1{ width:100%; float:left; padding: 5px;  }
#div_2204_01245_fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#div_2204_01245_fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Lya}{Ly\alpha}$
$\newcommand{\PSOJ}{PSO J083+11}$
$\newcommand{\cii}{\ion{C}{2}}$
$\newcommand{\mgii}{\ion{Mg}{2}}$
$\newcommand{\mgi}{\ion{Mg}{1}}$
$\newcommand{\oi}{\ion{O}{1}}$
$\newcommand{\siii}{\ion{Si}{2}}$
$\newcommand{\alii}{\ion{Al}{2}}$
$\newcommand{\feii}{\ion{Fe}{2}}$
$\newcommand{\civ}{\ion{C}{4}}$
$\newcommand{\hi}{\ion{H}{1}}$</div>

# Staring at the Shadows of Archaic Galaxies: and Metal Absorbers toward a Young z6 Weak-line Quasar

 [![arXiv](https://img.shields.io/badge/arXiv-2204.01245-b31b1b.svg)](https://arxiv.org/abs/2204.01245) _22 pages, 12 figures, and 1 table. Accepted for publication in The Astronomical Journal. We welcome the comments from the reader. Related paper: arXiv:2009.07784_

<mark>Irham Taufik Andika</mark>, et al. -- incl., <mark>Knud Jahnke</mark>, <mark>Eduardo Bañados</mark>, <mark>Sarah E. I. Bosman</mark>, <mark>Frederick B. Davies</mark> 

 **Abstract:** We characterize the$\Lya$halo and absorption systems toward$\PSOJ$, a unique$z=6.3401$weak-line quasar, using Gemini/GNIRS, Magellan/FIRE, and VLT/MUSE data.Strong absorptions by hydrogen and several metal lines (e.g.,$\cii$,$\mgii$, and$\oi$) are discovered in the spectrum, which indicates the presence of: (i) a proximate sub-damped$\Lya$(sub-DLA) system at$z=6.314$and (ii) a$\mgii$absorber at$z=2.2305$.To describe the observed damping wing signal, we model the$\Lya$absorption with a combination of a sub-DLA with the neutral hydrogen column density of$\log N_\mathrm{HI} = 20.03 \pm 0.30 \mathrm{cm^{-2}}$and absorption from the intergalactic medium with a neutral fraction of around 10\%.The sub-DLA toward$\PSOJ$has an abundance ratio of[C/O]$=-0.04 \pm 0.33$and metallicity of[O/H]$=-2.19 \pm 0.44$, similar to those of low-redshift metal-poor DLAs.These measurements suggest that the sub-DLA might truncate$\PSOJ$'s proximity zone size and complicate the quasar lifetime measurement.However, this quasar shows no sign of a$\Lya$halo in the MUSE datacube, where the estimated$1\sigma$limit of surface brightness is$2.76 \times 10^{-18} \mathrm{erg s^{-1} cm^{-2} arcsec^{-2}}$at aperture size of 1$\arcsec$, or equivalent to a$\Lya$luminosity of$\leq 43.46$erg s$^{-1}$.This non-detection, while being only weak independent evidence on its own, is at least consistent with a young quasar scenario, as expected for a quasar with a short accretion timescale.
<div id="div_2204_01245_fig1">

<img src="tmp_2204.01245/figures/fig_spec_model.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
		The MUSE + GNIRS spectrum of $\PSOJ$(black line) and its associated shot noise (grey line).
		The median composite spectrum of the low-redshift quasar analogs is shown as a cyan line with the $1\sigma$ dispersion around the median as a light blue region.
		The composite spectrum gives a decent fit to the main features observed in the $\PSOJ$ spectrum redward of the $\Lya$ emission.
		On the other hand, the PCA model to better predict the blue-side of the quasar spectrum and its $1\sigma$ dispersion are denoted with the red line and shaded region, respectively.
		The wavelength range and spectrum used in the PCA fit to predict $\Lya$ is shown with the red dashed line.
        The inset figure shows the zoom-in to the region around $\Lya$.
        The $\mgii$ $\lambda\lambda2796,2803$ doublet and $\mgi$ $\lambda2853$ absorptions at $z=2.2305$ are marked with blue dotted lines.
        The wavelength ranges highlighted with magenta ($\lambda_\mathrm{obs}=$ 8850--8997 \AA) and green ($\lambda_\mathrm{obs}=$ 9100--9200 \AA) colors are the region of interest for creating the $\Lya$ pseudo-narrowband and continuum images, respectively.
	 (*fig:spec_model_wide*)

</div>
<div id="div_2204_01245_fig2">

<img src="tmp_2204.01245/figures/fig_velo_plot_first.png" alt="Fig3.1" width="50%"/><img src="tmp_2204.01245/figures/fig_velo_plot_last.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** 
		The velocity stack of the sub-DLA metal absorption lines at $z=6.314$ observed in the continuum-normalized combined MUSE + FIRE spectrum.
		The data are shown as black lines with fitted models superposed in red.
		Note that the width of the red lines corresponds to the model uncertainty (see Section \ref{sec:absline_analysis}).
		Blue curves show a $1\sigma$ shot noise spectrum.
		We use grey lines to show the data in regions with high noise due to skylines.
		The absorptions from $\mgii$ $\lambda\lambda2796,2803$ doublet and  $\cii$ $\lambda1334$ are clearly seen while there is only a marginal detection of $\oi$ $\lambda1302$.
        Finally, the other associated metals like $\siii$, $\alii$, and $\feii$ which one would expect specifically for a strong absorber are not apparent in the spectrum.
	 (*fig:absline*)

</div>
<div id="div_2204_01245_fig3">

<img src="tmp_2204.01245/figures/fig_halo_flux.png" alt="Fig1" width="100%"/>

**Figure 1. -** The azimuthally averaged radial light profile of $\PSOJ$'s $\Lya$ emission measured from the PSF-subtracted pseudo-narrowband image.
    Tentative detection of the halo flux within a 0$\farcs$6 aperture radius is observed, but there are no apparent extended $\Lya$ emissions beyond this radius.
     (*fig:halo_flux*)

</div>

# Debugging papers

In [10]:
raise NotImplementedError("Manual Stop")

NotImplementedError: Manual Stop

In [ ]:
import importlib
importlib.reload(latex)
which = "2204.01245"
paper_id = f'{which:s}'
folder = f'tmp_{paper_id:s}'
doc = latex.LatexDocument(folder)    

In [ ]:
from TexSoup import TexSoup
import re

def bracket_error(source: str):
    """ Find problematic portions of the document """
    
    # Checking header
    begin_doc = next(re.finditer(r'\\begin\{document\}', doc.source)).span()[1]
    header = source[:begin_doc]
    text = header + r"\n\end{document}"

    try:
        print("Header check... ", end='')
        TexSoup(text)
        print("ok")
    except:
        print("error")
        raise RuntimeError("Error in the header")
        
    # Check the text per section until the end.
    # Do not stop and try them all.
    
    problematic_text = []
    
    sections = ([(0, begin_doc, 'until first section')] + 
                [(g.span()[0], g.span()[1], g.group()) for g in re.finditer(r'\\section\{.*\}', source)] +
                [(g.span()[0], g.span()[1], g.group()) for g in re.finditer(r'\\begin\{appendix\}', source)]
               )
    
    sections = sorted(sections, key=lambda x: x[0])
    
    prev_pos, prev_name = (0, 'header')
    for span, span_end, name in sections:
        text = source[prev_pos:span]
        if prev_pos > begin_doc:
            text = r"\n\begin{document}" + text + r"\n\end{document}"
        else:
            text = text + r"\n\end{document}"
        try:
            print(f"{prev_pos}:{prev_name}-->{span}:{name} check... ", end='')
            TexSoup(text, tolerance=1)  # allow not ending env
            print("ok")
            prev_pos = span
            prev_name = name
        except:
            print(f"error between {prev_pos} and {span}")
            problematic_text.append((prev_pos, source[prev_pos:span]))
            prev_pos = span
            prev_name = name
            # raise
    return problematic_text


def check_environment(text, offset=0):
    """ Check environment """
    env = re.compile(r"\\begin\{(?P<env>.*)\}(.*)\\end\{(?P=env)\}", re.DOTALL)

    for match in env.finditer(text):
        beg, end = match.span()
        beg += offset
        end += offset
        envname = match.groups()[0]
        try:
            latex.TexSoup(match.group())
        except:
            print(f"Error in {envname:s} between {beg} and {end}")
            return match.groups()[1], beg, end
    
#[check_environment(k) for k in bracket_error(doc.source)]
a = bracket_error(doc.source)
for ak in a:
    r = check_environment(ak[1], offset=ak[0])
    print(r[1], r[2])
    print(r[0])